# Self-Driving Car Engineer


## Project: **Finding Lane Lines on the Road**
***
In this project, it is aimed to find lane lines on the road to find region of motion for a self driving car.

## Import Packages

In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Helper Functions

In [3]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def derivative_slope(slopes,count=2, tol=1.0):
    """
    NOTE: This is the function that finds the different slopes according to their derivatives.
    In our case, we know there will be two lane lines. Therefore we defined count variable 2
    We know that there is a new slope if derivative of two slopes bigger than tol value
    """
    cnt = 0
    slope = []
    for i in range(1,len(slopes)):
        der_slope = slopes[i]-slopes[i-1]
        if abs(der_slope) > tol:
            slope.append(slopes[i])
            cnt += 1
            if cnt == count:
                break
    return slope


def draw_lines(img, lines, color=[255, 0, 0], thickness=10, tol=0.1):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """ 
    #We get y-length of the image 
    y_shape = img.shape[0]
    
    x = []
    y = []
    slopes_all = []
    
    #We find all slopes to find that how many different slopes are 
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            slopes_all.append(slope)

    slopes = derivative_slope(slopes_all)
  
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope_cur = (y2-y1)/(x2-x1) #Current slope is found
            for i in range(len(slopes)): #There will be x's and y's as many as the number of different slopes
                x.append([])
                y.append([])
                if abs(slope_cur-slopes[i]) < tol: #Every point of lines are classified according to their slopes
                    x[i].append([x1,x2])
                    y[i].append([y1,y2])
                

    row = len(slopes)
    
    for i in range(row): 
        x_arr = np.array(x[i]) #Lists are converted to arrays
        y_arr = np.array(y[i]) #Lists are converted to arrays
        x_all = x_arr.reshape(x_arr.shape[0]*x_arr.shape[1],) #Arrays are reshaped by (row*column,)
        y_all = y_arr.reshape(y_arr.shape[0]*y_arr.shape[1],) #Arrays are reshaped by (row*column,)
        [m,b] = np.polyfit(x_all,y_all,1) #A line which is 1 degree is found
        xstart = int(round((y_shape - b)/m)) #xstart point is found at the bottom of the image
        xend = int(round((np.min(y_all) - b)/m)) #xend point is found at the top of the line
        cv2.line(img, (xstart, y_shape), (xend, np.min(y_all)), color, thickness) #line is drawn

    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Build a Lane Finding For A Single Image

All test and processed images can be found in test_images directory

In [13]:
import os

def find_lane(image):   
    low_threshold = 50
    high_threshold = 150
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 40 #minimum number of pixels making up a line
    max_line_gap = 20   # maximum gap in pixels between connectable line segments
    last_image = np.copy(image)*0 # creating a blank to draw lines on
    imshape = image.shape
    
    gray = grayscale(image) #image is converted to grayscale
    blur = gaussian_blur(gray,5) #grayscale is blurred 
    canny_img = canny(blur, low_threshold, high_threshold) #edges are found
    vertices = np.array([[(140,imshape[0]),(940,imshape[0]),(540,330),(420,330)]]) #region is defined
    canny_reg = region_of_interest(canny_img,vertices) #region of interest is constructed


    line_image = hough_lines(canny_reg, rho, theta, threshold, min_line_len, max_line_gap) #lane lines are created
    last_image = weighted_img(line_image, image) #last version of image is formed
    return last_image

def process_images(path):
    image_files = os.listdir(path) #All image files in the directory are listed
    new_path = 'test_images_output/'
    for file in image_files: #Every image will be processed one by one
        image = plt.imread(path+file) #Image is read
        image_processed = find_lane(image) #The lane lines are found
        mpimg.imsave(new_path+file,image_processed) #Image is saved into output directory
        

process_images("test_images/")

## Test on Videos

We are now going to test our algorithm in two videos

In [15]:
import imageio
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [16]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(find_lane) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)
white_clip.reader.close()
white_clip.audio.reader.close_proc()
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:03<00:00, 68.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 3.6 s


In [18]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(find_lane)
%time yellow_clip.write_videofile(yellow_output, audio=False)
yellow_clip.reader.close()
yellow_clip.audio.reader.close_proc()
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:09<00:00, 68.52it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 10.3 s
